In [1]:
%load_ext autoreload
%autoreload 2
    
import tensorflow as tf
from tensorflow  import keras
from keras import metrics
import numpy as np

import librosa 
import matplotlib.pyplot as plt
from tqdm import tqdm
import h5py
import soundfile as sf
from scipy.signal import resample_poly
import librosa
from modAL.models import ActiveLearner
from scikeras.wrappers import KerasClassifier
import random as rand

from config import *
from util import DEFAULT_TOKENS
from models import build_resnet16
from preprocessing import AL_split

2024-09-30 12:45:24.313541: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 12:45:24.313577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 12:45:24.314641: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 12:45:24.321465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 12:45:25.112117: W tensorflow/comp

In [2]:
logmel_f = h5py.File(INTERMEDIATE / 'logmel.hdf5', 'r')
samples_f = h5py.File(INTERMEDIATE / '22sr_samples.hdf5', 'r')

X = np.load(INTERMEDIATE / 'logmel.npy')
Y = np.load(INTERMEDIATE / 'logmel_labels.npy')

In [3]:
batch = 32

# resnet 16
X_shape = (40, 107, 1)
model = build_resnet16(input_shape=X_shape)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[
                metrics.Recall(thresholds=0.5),
                metrics.Precision(thresholds=0.5),
                metrics.AUC(curve='pr', name='auc_pr')
              ])
classifier = KerasClassifier(model, batch_size=batch, verbose=2)

init, pool, test = AL_split(X, Y)
initial_X, initial_Y = init
pool_X, pool_Y = pool
test_X, test_Y = test
pool_X.shape, pool_Y.shape

((135462, 40, 107), (135462, 4))

In [4]:
currently_labelled = len(initial_X)
initial_ds_size = currently_labelled + len(pool_X)

def random_sampling(classifier, X_unlabelled, n_instances):
    i = np.random.choice(range(X_unlabelled.shape[0]), size=n_instances, replace=False)
    return i, X[i] 

learner = ActiveLearner(
    estimator=classifier,
    X_training=initial_X, # !!! TODO: use must larger initial labelled set (?)
    y_training=initial_Y,
    verbose=2,
    query_strategy=random_sampling 
) # trains on the initial ds

1059/1059 - 65s - 62ms/step - auc_pr: 0.0198 - loss: 0.0333 - precision: 0.0472 - recall: 0.0645


In [ ]:
from preprocessing import evaluation_dict
import pickle
from models import build_resnet16
from keras import metrics

n_queries = 200
query_size = int(pool_X.shape[0] / n_queries)

LB_metrics = []
trained_X = initial_X
trained_Y = initial_Y

print("=== RANDOM SAMPLING ===")
for idx in tqdm(range(n_queries)):
    print(f'Query no. {idx + 1}/{n_queries}')

    # query for instances
    query_indicies, query_instances = learner.query(pool_X, n_instances=query_size)
    
    # train on instances
    learner.teach(
        X=pool_X[query_indicies], y=pool_Y[query_indicies], 
        only_new=True, verbose=2
    )

    # get evaluation metrics
    print("evaluating ..")
    currently_labelled += query_size
    labelling_budget = currently_labelled / initial_ds_size 
    pred_Y = classifier.predict_proba(test_X, batch_size=batch, verbose=2)
    LB_metrics.append(
        (labelling_budget, evaluation_dict(pred_Y, test_Y)))

    # store trained on samples 
    trained_X = np.vstack((trained_X, pool_X[query_indicies]))
    trained_Y = np.vstack((trained_Y, pool_Y[query_indicies]))
    
    # remove queried instance from pool
    pool_X = np.delete(pool_X, query_indicies, axis=0)
    pool_Y = np.delete(pool_Y, query_indicies, axis=0)

    with open(INTERMEDIATE / 'AL' / 'RS_metrics.pkl', 'wb') as f:
        pickle.dump(LB_metrics, f)


=== RANDOM SAMPLING ===


  0%|                                                                                           | 0/200 [00:00<?, ?it/s]

Query no. 1/200
22/22 - 1s - 54ms/step - auc_pr: 0.4706 - loss: 0.0153 - precision: 0.5000 - recall: 0.2857
evaluating ..


2024-09-30 12:46:54.576724: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 724723840 exceeds 10% of free system memory.


1323/1323 - 18s - 14ms/step


...

Query no. 44/200
22/22 - 1s - 55ms/step - auc_pr: 0.2392 - loss: 0.0132 - precision: 0.0000e+00 - recall: 0.0000e+00
evaluating ..
1323/1323 - 18s - 13ms/step


/home/ec2-user/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{   'fast_trill_6khz': {   'auc_pr': 0.4672058859542629,
                           'f1': 0.1255088195386703,
                           'precision': 0.06821533923303835,
                           'recall': 0.7838983050847458},
    'nr_syllable_3khz': {   'auc_pr': 0.7934032163954059,
                            'f1': 0.7775467775467776,
                            'precision': 0.7727272727272727,
                            'recall': 0.7824267782426778},
    'triangle_3khz': {   'auc_pr': 0.03596782481428383,
                         'f1': 0.0,
                         'precision': 0.0,
                         'recall': 0.0},
    'upsweep_500hz': {   'auc_pr': 0.20262994962633393,
                         'f1': 0.0,
                         'precision': 0.0,
                         'recall': 0.0}}


 22%|██████████████████                                                                | 44/200 [15:22<54:42, 21.04s/it]

Query no. 45/200
22/22 - 1s - 55ms/step - auc_pr: 0.3925 - loss: 0.0182 - precision: 0.8333 - recall: 0.3571
evaluating ..


In [ ]:
pred_Y = classifier.predict_proba(test_X)

In [ ]:
from sklearn.metrics import average_precision_score, precision_recall_curve

for i in range(4):
    m = average_precision_score(test_Y[:, i], pred_Y[:, i])
    print(m)

def plot_pr(name, labels, predictions, **kwargs):
    prec, rec, _ = precision_recall_curve(labels, predictions)
    plt.plot(prec, rec, label=name, linewidth=2, **kwargs)
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.grid(True)

label_order = DEFAULT_TOKENS.keys()
for i, name in enumerate(label_order):
    plot_pr(name, test_Y[:, i], pred_Y[:, i])

In [ ]:
m = evaluation_dict(pred_Y, test_Y)

In [ ]:
from util import LABELS

x = [lb for lb, _ in LB_metrics]
for i, l in enumerate(LABELS):
    y = [m[l]['auc_pr'] for _, m in LB_metrics]
    plt.plot(y, x)
    plt.ylim(0, 1)
    plt.title(f'AP of {l} for Random Sampling')
    plt.show()